In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from uuid import uuid4
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
# PDF 로딩
path = "data//raw//청년 법령집.pdf"
loader = PyPDFLoader(file_path=path)
docs = loader.load()
print(type(docs))

for i in range(len(docs)):
    docs[i].page_content = (docs[i].page_content
                            .replace('발  간  등  록  번  호','')
                            .replace('법제처 국가법령정보센터', '')
                            .replace('법제처', '')
                            .replace('Korea Law Service Center', '')
                            .replace('국가법령정보센터', '')
                            .replace('\n', ' ')
                            .replace('\t', ' ')
                            )


<class 'list'>


In [3]:
### 평가 데이터로 사용할 context 추출
total_samples = 5

# index shuffle 후 total_samples만큼 context 추출
idx_list = list(range(len(docs)))
random.shuffle(idx_list)

eval_context_list = []
while len(eval_context_list) < total_samples:
    idx = idx_list.pop()
    context = docs[idx].page_content
    if len(context) > 50: # 50글자 이상인 text만 사용
        eval_context_list.append(context)

len(eval_context_list)

IndexError: pop from empty list

In [4]:
# user_input: 질문
# qa_context: 질문 답변 쌍을 만들 때 참고한 context
# retrieved_contexts: 검색된 문서의 내용은 실제 RAG 실행시 넣는다.
# response: 모델의 답변 - 실제 RAG 실행시 넣는다.
# reference: 정답
class EvalDatasetSchema(BaseModel):
    user_input: str = Field(..., title="질문(question)")
    qa_context: list[str] = Field(..., title="질문-답변 쌍을 만들 때 참조한 context.")
    reference: str = Field(..., title="질문의 정답(ground truth)")

eval_model = ChatOpenAI(model="gpt-4o")
parser = JsonOutputParser(pydantic_object=EvalDatasetSchema)

prompt_template = PromptTemplate.from_template(
    template=dedent("""
        당신은 RAG 평가를 위해 질문과 정답 쌍을 생성하는 인공지능 비서입니다.
        다음 [Context] 에 문서가 주어지면 해당 문서를 기반으로 {num_questions}개 질문-정답 쌍을 생성하세요. 
        
        질문과 정답을 생성한 후 아래의 출력 형식 GUIDE 에 맞게 생성합니다.
        질문은 반드시 [Context] 문서에 있는 정보를 바탕으로 생성해야 합니다. [Context]에 없는 내용을 가지고 질문-정답을 절대 만들면 안됩니다.
        질문은 간결하게 작성합니다.
        하나의 질문에는 한 가지씩만 내용만 작성합니다. 
        질문을 만들 때 "제공된 문맥에서", "문서에 설명된 대로", "주어진 문서에 따라" 또는 이와 유사한 말을 하지 마세요.
        정답은 반드시 [Context]에 있는 정보를 바탕으로 작성합니다. 없는 내용을 추가하지 않습니다.
        질문과 정답을 만들고 그 내용이 [Context] 에 있는 항목인지 다시 한번 확인합니다.
        생성된 질문-답변 쌍은 반드시 dictionary 형태로 정의하고 list로 묶어서 반환해야 합니다.
        질문-답변 쌍은 반드시 {num_questions}개를 만들어야 합니다.

        출력 형식: {format_instructions}
                    
        [Context]
        {context}
        """
    ),
    partial_variables={"format_instructions":parser.get_format_instructions()}
)

eval_dataset_generator = prompt_template | eval_model | parser

In [5]:
# eval_context_list
eval_data_list = []
num_questions = 5

for context in eval_context_list:
    _eval_data_list = eval_dataset_generator.invoke({"context":context, "num_questions":num_questions})
    eval_data_list.extend(_eval_data_list)

eval_df = pd.DataFrame(eval_data_list)
eval_df.shape

In [7]:
eval_df

,user_input,qa_context,reference
0,광양시 청년일자리 창출 조례의 목적은 무엇인가?,[광양시 청년일자리 창출 조례는 지역경제 활성화와 사회 안정에 이바지하기 위하여 청...,광양시 지역경제 활성화와 사회 안정에 이바지하기 위하여 청년일자리 창출 촉진에 관한...
1,조례에서 '청년'의 정의는 무엇인가?,[조례에서 '청년'이란 광양시에 주소를 가진 「청년고용촉진 특별법」 제2조 제1호의...,광양시에 주소를 가진 「청년고용촉진 특별법」 제2조 제1호의 사람이다.
2,광양시장은 청년일자리 창출을 위해 어떤 대책을 수립해야 하는가?,"[광양시장은 청년일자리 창출을 위하여 일자리 창출 및 인력수급 전망, 청년 미취업자...","일자리 창출 및 인력수급 전망, 청년 미취업자 실태조사 및 지원 방안, 직업지도, ..."
3,청년일자리 창출 계획에는 어떤 사항이 포함되어야 하는가?,"[청년일자리 창출 계획에는 목표와 방향, 교육ㆍ홍보, 채용행사 등 시책, 시 소재 ...","목표와 방향, 교육ㆍ홍보, 채용행사 등 시책, 시 소재 공공기관 및 기업 등과의 협..."
4,광양시장은 청년 미취업자 실태조사를 왜 실시하는가?,[광양시장은 청년 미취업자에 대한 실태조사를 실시하여 청년일자리 창출과 청년 고용촉...,청년일자리 창출과 청년 고용촉진을 위한 정책 및 창출계획 수립의 기초자료로 활용하기...
5,광양시 청년일자리 창출 촉진에 관한 조례는 언제 제정되었나요?,[부칙(제정 2016. 6. 29. 조례 제1433호)],2016년 6월 29일에 제정되었습니다.
6,광양시 청년일자리 창출 촉진에 관한 조례의 시행일은 언제인가요?,[제1조(시행일) 이 조례는 공포한 날부터 시행한다.],공포한 날부터 시행됩니다.
7,광양시 청년일자리 창출 촉진에 관한 조례에 따른 경비지원 보조금의 절차는 무엇을 따...,"[제1항에 따른 경비지원 보조금의 교부신청ㆍ절차ㆍ방법, 정산 등에 필요한 사항은 「...",「광양시 지방보조금 관리 조례」를 따릅니다.
8,2020년 개정된 조례에서 민간위탁 관련 조항이 어떻게 변경되었나요?,[제9조제2항 중 “「광양시 사무의 민간위탁 촉진 및 관리 조례」”를 “「광양시 사...,「광양시 사무의 민간위탁 촉진 및 관리 조례」를 「광양시 사무의 민간위탁 기본 조례...
9,조례 시행 전에 지원한 청년일자리 창출 촉진 사항은 어떻게 처리되나요?,[제2조(경과조치) 이 조례의 시행 전에 시장이 청년일자리 창출 촉진을 위하여 지원...,이 조례에 따른 것으로 봅니다.


In [8]:
# Vector Store
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
COLLECTION_NAME = "youth_policy"
PERSIST_DIRECTORY = "vector_store/chroma/youth_policy"

# vector store 연결
vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

In [9]:
# Chain 구성
# Prompt Template 생성
messages = [
        ("ai", """
        당신은 청년 정책 전문가입니다. 아래 문서의 내용으로만 답변해주세요.
        답변을 모르면 모른다고 대답해주세요.
        {context}"""),
        ("human", "{question}"),
]

prompt_template = ChatPromptTemplate(messages)

# Retriever 생성
retriever = vector_store.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k':10,
        'fetch_k':10,
        'lambda_mult':0.5
    }
)
model = ChatOpenAI(model="gpt-4o-mini")


def format_docs(src_docs:dict[str, list[Document]]) -> str:
    """list[Document]: Vector Store에서 검색한 context들에서 
    page_content만 추출해서 하나의 문자열로 합쳐서 반환"""
    docs = src_docs['context']
    return "\n\n".join([doc.page_content for doc in docs])


def str_from_documents(docs: list[Document]) -> list[str]:
    """list[Document]에서 page_content 값들만 추출한 list를 반환."""
    return [doc.page_content for doc in docs]


# LLM 응답 처리 chain.
rag_chain = (
    RunnablePassthrough() 
    | {
        "context": retriever, "question":RunnablePassthrough()
    } 
    | {
        "source_context" : itemgetter("context") | RunnableLambda(str_from_documents), 
        "llm_answer": {
            "context": RunnableLambda(format_docs), "question":itemgetter("question")
        } | prompt_template | model | StrOutputParser()   
    }
)


In [10]:
eval_df['user_input']

0                            광양시 청년일자리 창출 조례의 목적은 무엇인가?
1                                  조례에서 '청년'의 정의는 무엇인가?
2                   광양시장은 청년일자리 창출을 위해 어떤 대책을 수립해야 하는가?
3                       청년일자리 창출 계획에는 어떤 사항이 포함되어야 하는가?
4                          광양시장은 청년 미취업자 실태조사를 왜 실시하는가?
5                    광양시 청년일자리 창출 촉진에 관한 조례는 언제 제정되었나요?
6                   광양시 청년일자리 창출 촉진에 관한 조례의 시행일은 언제인가요?
7     광양시 청년일자리 창출 촉진에 관한 조례에 따른 경비지원 보조금의 절차는 무엇을 따...
8                2020년 개정된 조례에서 민간위탁 관련 조항이 어떻게 변경되었나요?
9               조례 시행 전에 지원한 청년일자리 창출 촉진 사항은 어떻게 처리되나요?
10                                        조례의 목적은 무엇인가?
11                                가족돌봄청소년ㆍ청년의 정의는 무엇인가?
12                                        시장의 책무는 무엇인가?
13                              기본계획에 포함되어야 할 사항은 무엇인가?
14                             시장이 추진할 수 있는 지원사업은 무엇인가?
15                     시장은 어떤 교육기관과 연계하여 청년을 지원할 수 있나요?
16                       청년일자리 창출을 위해 시장이 협력할 수 있는 기관은?
17                       시장이 청년일자리 창출 사업을 지원할 수 

In [11]:
# rag_chain에 평가 질문을 입력해서 context들과 모델답변을 응답 받아 eval_dataset(eval_df)에 추가.
context_list = []
response_list = []

for user_input in eval_df['user_input']:
    res = rag_chain.invoke(user_input)
    context_list.append(res['source_context'])
    response_list.append(res['llm_answer'])

In [12]:
response_list

['광양시 청년일자리 창출 촉진에 관한 조례의 목적은 「청년고용촉진 특별법」에 따라 광양시 지역경제 활성화와 사회 안정에 이바지하기 위하여 청년일자리 창출 촉진에 관한 사항을 정하는 것입니다.',
 "조례에서 '청년'의 정의는 일반적으로 19세 이상 39세 이하의 사람을 말합니다. 다만, 다른 법령이나 조례에서 청년의 연령을 다르게 적용하는 경우에는 그에 따릅니다. 일부 조례에서는 청년의 연령 범위를 18세 이상 45세 이하로 설정하기도 합니다.",
 '광양시장은 청년일자리 창출을 위해 다음과 같은 대책을 수립해야 합니다:\n\n1. 청년 고용 확대 및 일자리 질 향상: 청년 고용을 확대하고 일자리의 질을 향상시키기 위한 정책을 마련해야 합니다.\n\n2. 직업역량 강화 및 취업 지원: 청년 구직자의 직업역량을 강화하기 위한 교육 및 취업 지원 방안을 강구해야 합니다.\n\n3. 청년창업 지원: 청년 창업을 육성하기 위해 창업환경 개선과 안정적인 기반 조성을 위한 지원 대책을 마련해야 합니다.\n\n4. 민간기업과의 협력: 민간기업 등과 협력하여 청년 고용을 촉진할 수 있는 지표를 연구하고 개발하여 공공구매 등과 연계해야 합니다.\n\n5. 취약계층 청년 지원: 취약계층 청년의 자립과 노동시장 진입을 위한 구체적인 대책을 마련해야 합니다.\n\n이러한 대책을 통해 청년들이 안정적으로 일자리를 찾고, 경제적 자립을 이루도록 지원해야 합니다.',
 '청년일자리 창출 계획에는 다음 각 호의 사항이 포함되어야 합니다:\n\n1. 청년일자리 창출과 고용촉진을 위한 목표와 방향\n2. 청년일자리 창출을 위한 시책\n3. 청년일자리 창출을 위한 각종 지원 및 홍보에 관한 사항\n4. 기관, 단체, 기업 등과 협력에 관한 사항\n5. 그 밖에 시장이 필요하다고 인정하는 사항',
 '광양시장은 청년 미취업자에 대한 실태조사를 실시하여 청년일자리 창출과 청년 고용촉진을 위한 정책 및 창출계획 수립의 기초자료로 활용하기 위함입니다. 이 조사는 청년일자리 창출을 위한 효과적인 정책

In [13]:
# context 추가
eval_df["retrieved_contexts"] = context_list 

# 정답 추가
eval_df["response"] = response_list   

# Dataframe으로 부터 EvalDataset 생성
eval_dataset = EvaluationDataset.from_pandas(eval_df)

In [17]:
model_name = "gpt-4o-mini"
model = ChatOpenAI(model=model_name)
eval_llm = LangchainLLMWrapper(model)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
eval_embedding = LangchainEmbeddingsWrapper(embedding_model)


In [18]:
# 분당 토큰 리미트으로 평가별 나누어 실행행
run_config = RunConfig(
    timeout=360,     # LLM 호출 이후 최대 대기 시간 
    max_retries=20, # API 호출시 지정한 횟수만큼 재시도
    max_wait=360,   # 재시도 대기 시간(초)
    max_workers=1   # 병렬처리 worker 수.
)
metrics1 = [LLMContextRecall(llm=eval_llm)]
metrics2 = [LLMContextPrecisionWithReference(llm=eval_llm)]
metrics3 = [Faithfulness(llm=eval_llm)]
metrics4 = [AnswerRelevancy(llm=eval_llm, embeddings=eval_embedding)]

In [19]:
# context_recall
result1 = evaluate(dataset=eval_dataset, metrics=metrics1, run_config=run_config)
result1

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

{'context_recall': 0.8800}

In [20]:
# llm_context_precision_with_reference
result2 = evaluate(dataset=eval_dataset, metrics=metrics2, run_config=run_config)
result2

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

{'llm_context_precision_with_reference': 0.8037}

In [21]:
# faithfulness
result3 = evaluate(dataset=eval_dataset, metrics=metrics3, run_config=run_config)
result3

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

{'faithfulness': 0.9233}

In [22]:
# answer_relevancy
result4 = evaluate(dataset=eval_dataset, metrics=metrics4, run_config=run_config)
result4

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

{'answer_relevancy': 0.3105}